In [9]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
import time

# ChromeDriver 값과 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")  # 해산도를 높어서 축소 효과를 낼다.
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/search/%EC%A7%9C%EC%A7%9C%EB%A3%A8%20%EA%B0%95%EB%82%A8/place/1011242047?c=15.00,0,0,0,dh&placePath=%3Fentry%253Dbmp')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 정보 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색
    rating_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
    )
    rating_text = rating_element.text
    rating = rating_text.replace("별점", "").strip()

    # 방문자 리뷰 검색
    visitor_review_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
    )
    visitor_review = visitor_review_element.text.split("리뷰")[1].strip()

    # 블로그 리뷰 검색
    blog_review_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
    )
    blog_review = blog_review_element.text.split("리뷰")[1].strip()

    # 결과 출력
    print("\uac00\uac8c \uc774\ub984:", store_name)
    print("\uc5c5종 \uce74\ud14c\uace0\ub9ac:", category)
    print("\ubcc4\uc810:", rating)
    print("\ubc29\ubb38\uc790 \ub9ac\ubdf0:", visitor_review)
    print("\ube14\ub85c\uadf8 \ub9ac\ubdf0:", blog_review)

    # '리뷰' 탭 클릭하기
    try:
        review_tab_element = WebDriverWait(driver, 15).until(
            EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
        )
        review_tab_element.click()
        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 리뷰 총 개수 가져오기
    print("\n리뷰 총 개수 가져오기")
    try:
        review_count_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
        )
        total_reviews = int(review_count_element.text.replace(',', ''))
        print("총 리뷰 수:", total_reviews)
    except Exception as e:
        print("리뷰 총 개수 가져오기 오류:", e)

    review_count = 0
    reviews = []
    review_text_set = set()  # 중복 확인을 위한 set
    no_more_button_count = 0  # 더보기 버튼이 없을 때의 카운트

    while review_count < min(total_reviews, 1000):  # 최대 1000개 리뷰 수집
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        total_elements_on_page = len(review_elements)

        for i in range(review_count + 1, total_elements_on_page + 1):
            retry_count = 0
            while retry_count < 3:  # 최대 3번 재시도
                try:
                    # 리뷰 리스트를 매번 새로 가져옴
                    review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
                    current_review_element = review_elements[i - 1]

                    # 스크롤을 조정하여 요소가 화면에 나타나도록 함
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", current_review_element)
                    time.sleep(0.1)  # 스크롤 후 약간의 지연을 줌

                    # 리뷰 텍스트, 날짜 경로
                    review_text_xpath_with_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[5]/a[1]'
                    review_date_xpath_with_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[7]/div[2]/div/span[1]/span[2]'
                    review_text_xpath_without_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[4]/a'
                    review_date_xpath_without_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[6]/div[2]/div/span[1]/span[2]'

                    # 사진이 있을 때
                    try:
                        review_text_element = driver.find_element(By.XPATH, review_text_xpath_with_photo)
                        review_date_element = driver.find_element(By.XPATH, review_date_xpath_with_photo)
                    except NoSuchElementException:
                        # 사진이 없을 때
                        review_text_element = driver.find_element(By.XPATH, review_text_xpath_without_photo)
                        review_date_element = driver.find_element(By.XPATH, review_date_xpath_without_photo)

                    review_text = review_text_element.text.strip()
                    review_date = review_date_element.text.strip()

                    # 중복 확인 및 저장
                    if review_text and (review_text, review_date) not in review_text_set:
                        reviews.append((review_date, review_text))  # 사용자 이름 제외
                        review_text_set.add((review_text, review_date))  # 중복 확인을 위해 텍스트와 날짜 추가
                        review_count += 1
                        print(f"리뷰 {review_count}: {review_text} ({review_date})\n")
                    else:
                        print(f"중복된 리뷰 발견, 저장하지 않음: {review_text}")
                    break  # 성공하면 루프 탈출
                except (NoSuchElementException, TimeoutException, StaleElementReferenceException) as e:
                    retry_count += 1
                    print(f"리뷰 {review_count + 1}: 리뷰 텍스트 또는 날짜 없음 - 재시도 {retry_count}/{3}")
                    time.sleep(0.5)  # 재시도 전 약간의 지연

        # 하단 '더보기' 버튼이 있으면 클릭하고, 없으면 스크롤 내리기만 진행
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li'))
            )
            print("하단 '더보기' 버튼 클릭 완료")
            no_more_button_count = 0  # 버튼 클릭 시 카운트 초기화
        except (NoSuchElementException, TimeoutException):
            # 더보기 버튼이 없으면 스크롤을 내려 추가 리뷰 로드
            driver.execute_script("window.scrollBy(0, 300);")  # 조금씩 스크롤
            time.sleep(1)
            print("하단 '더보기' 버튼이 없어 스크롤 다운 진행")
            no_more_button_count += 1

        # 더보기 버튼이 5번 연속으로 없으면 저장하고 종료
        if no_more_button_count >= 5:
            print("하단 '더보기' 버튼이 5번 연속으로 없어져서 리뷰를 저장합니다.")
            break

    print(f"총 {review_count}개의 리뷰를 가져왔습니다.")

except Exception as e:
    # 예외가 발생할 경우 메시지 출력
    print(f"오류 발생: {e}")

# CSV 파일로 저장
if reviews:
    # 'reviews' 리스트의 각 항목이 ('Date', 'Review') 2개의 요소로 되어있다는 것을 확실히 합니다.
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews_추천순.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")

가게 이름: 짜짜루
업종 카테고리: 중식당
별점: 4.26
방문자 리뷰: 104
블로그 리뷰: 4
리뷰 탭 클릭 완료

리뷰 총 개수 가져오기
총 리뷰 수: 51
리뷰 1: 맛있고 양도 많아서 점심에 종종 먹으러 갑니다 (2024년 2월 13일 화요일)

리뷰 2: 짜장면 맛있어요 (2024년 5월 2일 목요일)

리뷰 3: 와.무슨.짱뽕하나 먹길.20뷴 이상 기달리고
기달리고 사장 아쥼마 완전 불친절 ㅠㅠㅠ
두번 다시.안가요 ㅠㅠ
ㅠㅠ
짬뽕인지 고추장 라면인지.ㅠ (2024년 4월 6일 토요일)

리뷰 4: 짜장면 맛있어요
양이 정말 푸짐 (2024년 1월 10일 수요일)

리뷰 5: 건물 지하1층 위치 (2024년 1월 9일 화요일)

리뷰 6: 맛이 전반적으로 깔끔하고 맛있어요!!
강남에서 가격도 저렴한 편입니다ㅎㅎ
근처여서 자주 올 것 같아요!! (2023년 10월 20일 금요일)

리뷰 7: 굿 (2024년 3월 20일 수요일)

리뷰 8: 오래된 상가 내 식당 (2023년 12월 21일 목요일)

리뷰 9: 양이 많아요 (2024년 2월 21일 수요일)

리뷰 10: 짜장면이 진짜 맛있어요 (2023년 12월 20일 수요일)

하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
하단 '더보기' 버튼 클릭 완료
리뷰 11: 맛있었어요.^^ 짬뽕 국물 진짜 맛있어요 :) (2023년 8월 12일 토요일)

리뷰 12: 음식은 맛있어요~~~ 근데 탕수육 양 많다고 하셨는데, 가격대비 가성비가 별로였어요.. (2023년 10월 17일 화요일)

리뷰 13: 탕수육 바삭하면서 부드럽고 볶음밥 지대로에 짬뽕국물 칼칼하고 매콤하고 짜지않아 좋았어요. 가격도 이 정도면 매우 만족합니다. 추천해요 (2023년 9월 18일 월요일)

리뷰 14: 몇일전 혼자먹으러 왔다가 맛있어서 같이왔어요.
오늘은 군만두도 시켰는데 정말 맛있었어요ㅠㅠ (2023년 9월 21일 목요일)

리뷰 15: 가성비ㅜ

In [10]:
import os
print("Current working directory:", os.getcwd())


Current working directory: C:\fintech_service\final_project


# 큐 뭐시기

In [13]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
import time

# ChromeDriver 값과 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument("--disable-gpu")
options.add_argument("--window-size=2560,1440")  # 해산도를 높어서 축소 효과를 낼다.
options.add_experimental_option("excludeSwitches", ["enable-logging", "enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

# ChromeDriver 초기화
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

# 네이버 지도 웹사이트로 이동
driver.get('https://map.naver.com/p/search/%ED%81%90%EB%AE%AC%EB%9F%AC%EC%8A%A4/place/1876924877?c=15.00,0,0,0,dh&isCorrectAnswer=true')

# 페이지 로드 대기
WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "entryIframe")))

try:
    # entryIframe이 로드될 때까지 대기 후 전환
    WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "entryIframe")))

    # 가게 정보 검색
    store_name_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[1]"))
    )
    store_name = store_name_element.text

    # 업종 카테고리 검색
    category_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='_title']/div/span[2]"))
    )
    category = category_element.text

    # 별점 검색
    rating_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/div[2]/div[1]/div[2]/span[1]"))
    )
    rating_text = rating_element.text
    rating = rating_text.replace("별점", "").strip()

    # 방문자 리뷰 검색
    visitor_review_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[2]/a"))
    )
    visitor_review = visitor_review_element.text.split("리뷰")[1].strip()

    # 블로그 리뷰 검색
    blog_review_element = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='app-root']/div/div/div/div[2]/div[1]/div[2]/span[3]/a"))
    )
    blog_review = blog_review_element.text.split("리뷰")[1].strip()

    # 결과 출력
    print("가게 이름:", store_name)
    print("업종 카테고리:", category)
    print("별점:", rating)
    print("방문자 리뷰:", visitor_review)
    print("블로그 리뷰:", blog_review)

    # '리뷰' 탭 클릭하기
    try:
        # 기존 XPATH로 탭을 클릭하지 못할 경우 새로운 위치에서 클릭 시도
        try:
            review_tab_element = WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.XPATH, "//span[text()='리뷰']"))
            )
            review_tab_element.click()
        except TimeoutException:
            # 새 위치에서 '리뷰' 탭 클릭 시도
            review_tab_element_alternative = WebDriverWait(driver, 15).until(
                EC.element_to_be_clickable((By.XPATH, "//*[@id='app-root']/div/div/div/div[4]/div/div/div/div/a[4]"))
            )
            review_tab_element_alternative.click()

        WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em')))
        print("리뷰 탭 클릭 완료")
    except Exception as e:
        print("리뷰 탭 클릭 오류:", e)

    # 리뷰 총 개수 가져오기
    print("\n리뷰 총 개수 가져오기")
    try:
        review_count_element = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/h2/div[1]/em'))
        )
        total_reviews = int(review_count_element.text.replace(',', ''))
        print("총 리뷰 수:", total_reviews)
    except Exception as e:
        print("리뷰 총 개수 가져오기 오류:", e)

    review_count = 0
    reviews = []
    review_text_set = set()  # 중복 확인을 위한 set
    no_more_button_count = 0  # 더보기 버튼이 없을 때의 카운트

    while review_count < min(total_reviews, 1000):  # 최대 1000개 리뷰 수집
        review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
        total_elements_on_page = len(review_elements)

        for i in range(review_count + 1, total_elements_on_page + 1):
            retry_count = 0
            while retry_count < 3:  # 최대 3번 재시도
                try:
                    # 리뷰 리스트를 매번 새로 가져옴
                    review_elements = driver.find_elements(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li')
                    current_review_element = review_elements[i - 1]

                    # 스크롤을 조정하여 요소가 화면에 나타나도록 함
                    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", current_review_element)
                    time.sleep(0.1)  # 스크롤 후 약간의 지연을 줌

                    # 리뷰 텍스트, 날짜 경로
                    review_text_xpath_with_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[5]/a[1]'
                    review_date_xpath_with_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[7]/div[2]/div/span[1]/span[2]'
                    review_text_xpath_without_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[4]/a'
                    review_date_xpath_without_photo = f'//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li[{i}]/div[6]/div[2]/div/span[1]/span[2]'

                    # 사진이 있을 때
                    try:
                        review_text_element = driver.find_element(By.XPATH, review_text_xpath_with_photo)
                        review_date_element = driver.find_element(By.XPATH, review_date_xpath_with_photo)
                    except NoSuchElementException:
                        # 사진이 없을 때
                        review_text_element = driver.find_element(By.XPATH, review_text_xpath_without_photo)
                        review_date_element = driver.find_element(By.XPATH, review_date_xpath_without_photo)

                    review_text = review_text_element.text.strip()
                    review_date = review_date_element.text.strip()

                    # 중복 확인 및 저장
                    if review_text and (review_text, review_date) not in review_text_set:
                        reviews.append((review_date, review_text))  # 사용자 이름 제외
                        review_text_set.add((review_text, review_date))  # 중복 확인을 위해 텍스트와 날짜 추가
                        review_count += 1
                        print(f"리뷰 {review_count}: {review_text} ({review_date})\n")
                    else:
                        print(f"중복된 리뷰 발견, 저장하지 않음: {review_text}")
                    break  # 성공하면 루프 탈출
                except (NoSuchElementException, TimeoutException, StaleElementReferenceException) as e:
                    retry_count += 1
                    print(f"리뷰 {review_count + 1}: 리뷰 텍스트 또는 날짜 없음 - 재시도 {retry_count}/{3}")
                    time.sleep(0.5)  # 재시도 전 약간의 지연

        # 하단 '더보기' 버튼이 있으면 클릭하고, 없으면 스크롤 내리기만 진행
        try:
            bottom_more_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div[2]/div/a'))
            )
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", bottom_more_button)
            driver.execute_script("arguments[0].click();", bottom_more_button)
            WebDriverWait(driver, 5).until(
                EC.presence_of_all_elements_located((By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[3]/div[3]/div/ul/li'))
            )
            print("하단 '더보기' 버튼 클릭 완료")
            no_more_button_count = 0  # 버튼 클릭 시 카운트 초기화
        except (NoSuchElementException, TimeoutException):
            # 더보기 버튼이 없으면 스크롤을 내려 추가 리뷰 로드
            driver.execute_script("window.scrollBy(0, 300);")  # 조금씩 스크롤
            time.sleep(1)
            print("하단 '더보기' 버튼이 없어 스크롤 다운 진행")
            no_more_button_count += 1

        # 더보기 버튼이 5번 연속으로 없으면 저장하고 종료
        if no_more_button_count >= 5:
            print("하단 '더보기' 버튼이 5번 연속으로 없어져서 리뷰를 저장합니다.")
            break

    print(f"총 {review_count}개의 리뷰를 가져왔습니다.")

except Exception as e:
    # 예외가 발생할 경우 메시지 출력
    print(f"오류 발생: {e}")

# CSV 파일로 저장
if reviews:
    # 'reviews' 리스트의 각 항목이 ('Date', 'Review') 2개의 요소로 되어있다는 것을 확실히 합니다.
    reviews_df = pd.DataFrame(reviews, columns=['Date', 'Review'])
    reviews_df.index = range(1, len(reviews) + 1)
    csv_filename = f"{store_name}_reviews_추천순.csv"
    reviews_df.to_csv(csv_filename, index_label="Index", encoding='utf-8-sig')
    print(f"리뷰 데이터가 '{csv_filename}' 파일로 저장되었습니다.")
else:
    print("수집된 리뷰가 없습니다.")


오류 발생: Message: 
Stacktrace:
	GetHandleVerifier [0x006EEBD3+24307]
	(No symbol) [0x00678D74]
	(No symbol) [0x0055C323]
	(No symbol) [0x0059DC86]
	(No symbol) [0x0059DECB]
	(No symbol) [0x005DB9F2]
	(No symbol) [0x005BFED4]
	(No symbol) [0x005D9579]
	(No symbol) [0x005BFC26]
	(No symbol) [0x0059219C]
	(No symbol) [0x0059311D]
	GetHandleVerifier [0x00998D93+2818227]
	GetHandleVerifier [0x009F542E+3196750]
	GetHandleVerifier [0x009ED9D2+3165426]
	GetHandleVerifier [0x0078DA70+675216]
	(No symbol) [0x00681B3D]
	(No symbol) [0x0067EA18]
	(No symbol) [0x0067EBB5]
	(No symbol) [0x00671640]
	BaseThreadInitThunk [0x763FFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x7743809E+286]
	RtlGetAppContainerNamedObjectPath [0x7743806E+238]

리뷰 데이터가 '큐뮬러스_reviews_추천순.csv' 파일로 저장되었습니다.
